In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image
import time


In [2]:
wd = webdriver.Chrome()

In [4]:
kw_dict = {
    'AU': ['australian+banknote', 'australian+bank+note', 'australia+banknote', 'AU+banknote', 'AU+bank+note',
           'australian+currency+note', 'AU+currency+note', 'australian+dollar+note']
}

In [ ]:
def get_images_from_google(wd, delay, max_images, curr='AU'):
    def scroll_down(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)
    for kw in kw_dict[curr]:
        url = f"https://www.google.com/search?q=australian+bank+notes&tbm=isch&ved=2ahUKEwicnLqB5-qBAxU6a2wGHVOBDJgQ2-cCegQIABAA&oq={kw}&gs_lcp=CgNpbWcQAzIFCAAQgAQyBQgAEIAEMgYIABAFEB4yBggAEAUQHjIGCAAQBRAeMgYIABAFEB4yBwgAEBgQgAQyBwgAEBgQgAQyBwgAEBgQgAQyBwgAEBgQgAQ6CggAEIoFELEDEEM6CAgAEIAEELEDOgcIABCKBRBDOgYIABAIEB5Q9wtYxSJgniNoAHAAeACAAeYBiAHnGZIBBjAuMTguM5gBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=uugkZZzXObrWseMP04KywAk&bih=817&biw=1707&rlz=1C1CHBF_enCA918CA918#imgrc=1RjR-xtqYvl8cM"
        wd.get(url)

        image_urls = set()
        skips = 0

        while len(image_urls) + skips < max_images:
            scroll_down(wd)

            thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

            for img in thumbnails[len(image_urls) + skips:max_images]:
                try:
                    img.click()
                    time.sleep(delay)
                except:
                    continue

                images = wd.find_elements(By.CLASS_NAME, "pT0Scc")
                for image in images:
                    if image.get_attribute('src') in image_urls:
                        max_images += 1
                        skips += 1
                        break

                    if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                        image_urls.add(image.get_attribute('src'))
                        print(f"Found {len(image_urls)}")
                        
    return image_urls

In [ ]:
def download_image(download_path, url, file_name):
	try:
		image_content = requests.get(url).content
		image_file = io.BytesIO(image_content)
		image = Image.open(image_file)
		file_path = download_path + file_name

		with open(file_path, "wb") as f:
			image.save(f, "JPEG")

		print("Success")
	except Exception as e:
		print('FAILED -', e)


In [ ]:
urls = get_images_from_google(wd, 0.2, 1000)

for i, url in enumerate(urls):
	download_image("imgs\\", url, str(i) + ".jpg")